## Chatbots assistentes virtuais

Vamos começar treinando o nosso modelo do Rasa-nlu. Essa parte não vai ser muito aprofundada, pois já vimos o processo antes. 

In [1]:
import warnings
warnings.simplefilter("ignore")


from rasa_nlu.config import RasaNLUConfig
from rasa_nlu.model import Trainer
from rasa_nlu.converters import load_data

training_data = load_data("training_data.json")

pipeline = [
    "nlp_spacy",
    "tokenizer_spacy",
    "ner_crf"
]

config = RasaNLUConfig(cmdline_args={"pipeline": pipeline})

trainer = Trainer(config)
model_directory = trainer.persist('./models/')

print(f"Saving metadata at {model_directory}")

interpreter = trainer.train(training_data)

Saving metadata at ./models/model_20201002-120618


In [2]:
print(interpreter.parse("I'd like to go to a cheap Mexican restaurant"))

{'intent': {'name': '', 'confidence': 0.0}, 'entities': [{'start': 20, 'end': 25, 'value': 'cheap', 'entity': 'low-price', 'extractor': 'ner_crf'}, {'start': 26, 'end': 33, 'value': 'Mexican', 'entity': 'cousine', 'extractor': 'ner_crf'}], 'text': "I'd like to go to a cheap Mexican restaurant"}


Muitas vezes os chatbots precisam interagir com bases de dados, elas podem estar em bases SQL ou em APIs.

Vamos agora supor que, num mundo pós COVID-19, você se viaja para Bengaluru, na Índia. Vamos fazer um chatbot que te ajude a encontrar restaurantes nessa localidade!

A base de dados utilizada adapatada do [Zomato Bangalore Restaurants](https://www.kaggle.com/himanshupoddar/zomato-bangalore-restaurants)

In [3]:
import sqlite3
import random

Com o sqlite, o processo para se conectar a uma database em sql é:
``` Python
conn = sqlite3.connect(database.db)
c = conn.cursor()
c.execute("SELECT * FROM  ... (query em SQL)")
```

O primeiro passo então, é criar uma função que recebe os parametros que são extraidos com o rasa e cria uma query em SQL

In [4]:
def find_restaurants(params):
    # Create the base query
    query = 'SELECT name, address,rate_clean FROM restaurants_bengaluru'
    # Add filter clauses for each of the parameters
    filters = []
    t = []
    for ent in params['entities']:
        if ent['entity'] == 'cousine':
            filters = filters + ["{}=?".format(ent['value'])]
            t.append(1)
        else:
            filters = filters + ["price_classification=?"]
            t.append(ent['entity'])
    query += " WHERE " + " and ".join(filters)
    # Create the tuple of values
    t = tuple(t)
    # Open connection to DB
    conn = sqlite3.connect("nlp.db")
    # Create a cursor
    c = conn.cursor()
    # Execute the query
    c.execute(query, t)
    # Return the results
    return c.fetchall()

In [5]:
find_restaurants(interpreter.parse("I'd like to go to a fancy Mexican restaurant"))

[('San Churro Cafe',
  '1112, Next to KIMS Medical College, 17th Cross, 2nd Stage, Banashankari, Bangalore',
  '3.8'),
 ('San Churro Cafe',
  '1112, Next to KIMS Medical College, 17th Cross, 2nd Stage, Banashankari, Bangalore',
  '3.8'),
 ('Peppy Peppers',
  '942, 3rd Floor, 21st Main Road, 2nd Stage, Banashankari, Bangalore',
  '4.2'),
 ('Gustoes Beer House',
  '10, 30th Main, Deve Gowda Petrol Bunk Road, 3rd Stage, Banashankari, Bangalore',
  '4.1')]

Agora, vamos criar uma função que recebe a mensagem de texto do usuário e cria uma resposta do bot

In [6]:
def response(message):
    
    not_found_responses = ['Sorry, I could find nothing like that :/', "I can't find such thing, sorry",
                           'Maybe try again later']
    
    found_response_1 = ["There is this amazing place {0} at {1}, it is rated {2} in Zomato's website",
                     "Your lucky, I found this palce {0} at {1}! Rated {2}"]
    
    found_response_more = ["There are some options like that, here's one you might like: {0} - {1}.",
                          "I found this place {0} at {1} and it is rated {2}, but i have more options :)"]
    
    interpreter_message = interpreter.parse(message)
    possibilities = find_restaurants(interpreter_message)
    if len(possibilities)==0:
        response = random.choice(not_found_responses)
    elif len(possibilities) == 1:
        answer = random.choice(found_response_1)
        response = answer.format(*possibilities[0])
    else:
        answer = random.choice(found_response_more)
        restaurant = random.choice(possibilities)
        response = answer.format(*restaurant)
    return response
        

In [7]:
response("I'd like to go to a cheap Burger restaurant")

"There are some options like that, here's one you might like: Burger King - 59/3, Ground & First Floor, West Anjaneya Temple Road, Ward 49, Gandhi Bazaar, Basavanagudi, Bangalore."

In [9]:
response("I'd like to go to a cheap Mexican restaurant")

"I can't find such thing, sorry"

In [10]:
response("I'd like to go to a cheap Indian restaurant")

'I found this place Udupi Upahar at No.1585, 22nd Main,Banashankari 1st Stage and it is rated 3.8, but i have more options :)'